In [2]:
import arxiv
import requests
import fitz  # PyMuPDF
import os

# Directory to save PDFs
pdf_dir = "arxiv_papers"
os.makedirs(pdf_dir, exist_ok=True)

# Search for papers
search = arxiv.Search(
    query="Large Language Model",  # You can change the category or keyword
    max_results=100,
    sort_by=arxiv.SortCriterion.Relevance
)

def download_pdf(pdf_url, filename):
    response = requests.get(pdf_url)
    with open(filename, 'wb') as f:
        f.write(response.content)

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    full_text = ""
    for page in doc:
        full_text += page.get_text()
    return full_text

papers_text = []

for result in search.results():
    pdf_path = os.path.join(pdf_dir, f"{result.entry_id.split('/')[-1]}.pdf")
    if not os.path.exists(pdf_path):
        print(f"Downloading {result.title}")
        download_pdf(result.pdf_url, pdf_path)
    else:
        print(f"PDF already downloaded: {pdf_path}")

    print(f"Extracting text from {pdf_path}")
    text = extract_text_from_pdf(pdf_path)
    papers_text.append({
        "title": result.title,
        "text": text
    })

print(f"Extracted text from {len(papers_text)} papers")


/tmp/ipykernel_59789/546661828.py:31: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


Extracting text from arxiv_papers/2306.07377v1.pdf
Extracting text from arxiv_papers/2202.03371v1.pdf
Extracting text from arxiv_papers/2305.06530v1.pdf
Extracting text from arxiv_papers/2408.10441v1.pdf
Extracting text from arxiv_papers/2404.09579v1.pdf
Extracting text from arxiv_papers/2405.18774v1.pdf
Extracting text from arxiv_papers/2205.07634v1.pdf
Extracting text from arxiv_papers/2408.15040v2.pdf
Extracting text from arxiv_papers/2303.00077v1.pdf
Extracting text from arxiv_papers/2305.13267v1.pdf
Extracting text from arxiv_papers/2104.10441v1.pdf
Extracting text from arxiv_papers/2305.13782v1.pdf
Extracting text from arxiv_papers/2501.09653v1.pdf
Extracting text from arxiv_papers/2311.04329v2.pdf
Extracting text from arxiv_papers/2010.12858v2.pdf
Extracting text from arxiv_papers/2402.01065v1.pdf
Extracting text from arxiv_papers/2504.07315v2.pdf
Extracting text from arxiv_papers/2404.04748v2.pdf
Extracting text from arxiv_papers/2504.18572v1.pdf
Extracting text from arxiv_pape

In [3]:
import os
import fitz  # PyMuPDF
from langchain.schema import Document

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def load_documents_from_folder(folder_path):
    documents = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            path = os.path.join(folder_path, filename)
            text = extract_text_from_pdf(path)
            documents.append(Document(page_content=text, metadata={"source": filename}))
    return documents

# Example
pdf_folder = "arxiv_papers"
documents = load_documents_from_folder(pdf_folder)
print(f"Loaded {len(documents)} documents.")


Loaded 100 documents.


In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
docs = text_splitter.split_documents(documents)
print(f"Split into {len(docs)} chunks.")


Split into 13882 chunks.


In [15]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db = FAISS.from_documents(docs, embedding_model)
db.save_local("top100_papers_vector_db")
print("Vector store saved.")


Vector store saved.


In [17]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model="llama3-70b-8192",
    openai_api_base="https://api.groq.com/openai/v1",
    openai_api_key="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"
)

db = FAISS.load_local(
    "top100_papers_vector_db",
    embedding_model,
    allow_dangerous_deserialization=True
)
retriever = db.as_retriever(search_kwargs={"k": 3})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)


In [19]:
query = "Who is Large Language Models ?"
result = qa_chain.invoke({"query": query})

print("Answer:")
print(result["result"])

print("\nSource documents:")
for doc in result["source_documents"]:
    print(doc.metadata["source"])


Answer:
The title "Talking About Large Language Models" and the text do not refer to a person or entity named "Large Language Models". Instead, "Large Language Models" is a term used to describe a type of artificial intelligence model that processes and generates human-like language.

Source documents:
2412.10291v1.pdf
2402.09748v1.pdf
2406.13138v2.pdf


In [4]:

class Matrix:
    def __init__(self,rows,cols,elements):
        self.rows=rows
        self.cols=cols
        self.elements=elements
    
    def print_rows(self):
        print(self.rows)
        
    def print_cols(self):
        print(self.cols)
        
    def print_elements(self):
        for i in range(self.rows):
            rows=self.elements[i*self.cols:(i+1)*self.cols]
            print(' '.join(map(str,rows)))
        
try:
    fl=input().strip()
    fl_2,total_elements=fl.split()
    rows,cols=map(int,fl_2.lower().split('x'))
    total_elements=int(total_elements)
    elements=[]
    while len(elements) < total_elements:
        elements += list(map(int,input().strip().split()))
        
    matrix=Matrix(rows,cols,elements)
    matrix.print_rows()
    matrix.print_cols()
    matrix.print_elements()

except Excepiton:
    print('invalid')

3
3
1 2 3
4 5 6
7 8 9


In [6]:
class Matrix:
    def __init__(self,rows,cols,elements):
        self.rows=rows
        self.cols=cols
        self.elements=elements
        
    def print_rows(self):
        print(self.rows)
        
    def print_cols(self):
        print(self.cols)
        
    def print_elements(self):
        for i in range(self.rows):
            rows=self.elements[i*self.cols:(i+1)*self.cols]
            print(' '.join(map(str,rows)))
        
try:
    fl=input().strip()
    fl_2,total_elements=fl.split()
    rows,cols=map(int,fl_2.lower().split('x'))
    total_elements=int(total_elements)
    elements=[]
    while len(elements) < total_elements:
        elements += list(map(int,input().strip().split()))
        
    matrix=Matrix(rows,cols,elements)
    matrix.print_rows()
    matrix.print_cols()
    matrix.print_elements()

except Excepiton:
    print('invalid')

3
3
1 2 3
4 5 6
7 8 9


In [3]:
class Triangle:
    def __init__(self, a, b, c):
        self.a=a
        self.b=b
        self.c=c
        
    def perimeter(self,a,b,c):
        return a+b+c
        
def main():
    a=int(input())
    b=int(input())
    c=int(input())
    triangle=Triangle(a,b,c)
    
    print(triangle.perimeter(a,b,c))
    
    
if __name__ == "__main__":
    main()

12


In [9]:
class Triangle:
    def calculate_area(self,base,height):
        return base * height // 2

base,height=6,6


tri=Triangle()
tri.calculate_area(base,height)


18